### Process Theil-Sen trend of temperature from ERA5 monthly aggregate

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.stats import theilslopes
import rasterio
from rasterio.transform import from_origin
from pathlib import Path

def process_theil_sen_regression(input_nc_file, aggregation='yearly', min_year=None, max_year=None):
    """
    Calculate Theil-Sen slope for temperature data and save results as GeoTIFF.
    
    Parameters:
    input_nc_file (str): Path to NetCDF file with temperature data.
    aggregation (str): 'yearly', 'seasonal', or 'quarterly'. Default is 'yearly'.
    min_year (int): Optional, minimum year for analysis.
    max_year (int): Optional, maximum year for analysis.
    
    Outputs:
    GeoTIFF files with the slopes, named based on the aggregation and time range.
    """
    ds = xr.open_dataset(input_nc_file, engine='netcdf4')

    if not np.issubdtype(ds['date'].dtype, np.datetime64):
        ds['date'] = pd.to_datetime(ds['date'].values, format='%Y%m%d')

    if aggregation == 'yearly':
        resampled = ds.resample(date='Y').mean()
        process_and_save(resampled, input_nc_file, 'yearly', min_year, max_year)

    elif aggregation == 'seasonal':
        seasons = {
            "DJF": resample_by_months(ds, ['12', '01', '02']),
            "MAM": resample_by_months(ds, ['03', '04', '05']),
            "JJA": resample_by_months(ds, ['06', '07', '08']),
            "SON": resample_by_months(ds, ['09', '10', '11'])
        }
        for season, data in seasons.items():
            process_and_save(data, input_nc_file, f'seasonal_{season}', min_year, max_year)

    elif aggregation == 'quarterly':
        quarters = {
            "JFM": resample_by_months(ds, ['01', '02', '03']),
            "AMJ": resample_by_months(ds, ['04', '05', '06']),
            "JAS": resample_by_months(ds, ['07', '08', '09']),
            "OND": resample_by_months(ds, ['10', '11', '12'])
        }
        for quarter, data in quarters.items():
            process_and_save(data, input_nc_file, f'quarterly_{quarter}', min_year, max_year)

    else:
        raise ValueError("Invalid aggregation. Choose 'yearly', 'seasonal', or 'quarterly'.")

def resample_by_months(ds, months):
    return ds.sel(date=ds['date'].dt.month.isin([int(m) for m in months])).resample(date='Y').mean()

def process_and_save(resampled, input_nc_file, aggregation, min_year=None, max_year=None):
    
    resampled['t2m'] = (resampled['t2m'] - 273.15) * 9 / 5 + 32
    years = resampled['date'].dt.year.values
    start_year = min_year if min_year else years.min()
    end_year = max_year if max_year else years.max()

    year_mask = (years >= start_year) & (years <= end_year)
    t2m_data = resampled['t2m'].values[year_mask, :, :]

    lat, lon = resampled['latitude'].values, resampled['longitude'].values
    slope_array = np.zeros((len(lat), len(lon)), dtype=np.float32)

    for i in range(len(lat)):
        for j in range(len(lon)):
            pixel_time_series = t2m_data[:, i, j]
            if not np.any(np.isnan(pixel_time_series)):
                slope, _, _, _ = theilslopes(pixel_time_series, years[year_mask])
                slope_array[i, j] = slope

    slope_array *= (end_year - start_year)
    transform = from_origin(np.min(lon), np.max(lat), np.abs(lon[1] - lon[0]), np.abs(lat[1] - lat[0]))

    meta = {
        'driver': 'GTiff',
        'height': slope_array.shape[0],
        'width': slope_array.shape[1],
        'count': 1,
        'dtype': 'float32',
        'crs': 'EPSG:4326',
        'transform': transform
    }

    output_file_name = f"{Path(input_nc_file).stem}_theilsen_{aggregation}_{start_year}_{end_year}.tif"
    output_tif = Path(input_nc_file).with_name(output_file_name)

    with rasterio.open(output_tif, 'w', **meta) as dst:
        dst.write(slope_array, 1)

    print(f"GeoTIFF saved as {output_file_name}")

# Example usage:
# calculate_theil_sen_and_save_tif('temperature_data.nc', aggregation='seasonal', min_year=1980)

ds = r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\Monthly_single_l\data_0.nc"
process_theil_sen_regression(ds, min_year=1950)

c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


GeoTIFF saved as data_0_theilsen_yearly_1950_2024.tif


### Process OLS trend from ERA5 monthly aggregate

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import from_origin
from pathlib import Path

def process_ols_regression(input_nc_file, aggregation='yearly', aggregation_spec=None, min_year=None, max_year=None):
    # Open the NetCDF file
    ds = xr.open_dataset(input_nc_file, engine='netcdf4')

    # Harmonize 'date' or 'valid_time' to ensure consistent date handling
    if not np.issubdtype(ds['date'].dtype, np.datetime64):
        ds['date'] = pd.to_datetime(ds['date'].values, format='%Y%m%d')

    # Aggregation type logic
    if aggregation == 'yearly':
        resampled = ds.resample(date='Y').mean()
        output_suffix = f"yearly_{min_year}_{max_year}"
        process_and_save(resampled, input_nc_file, output_suffix, min_year, max_year)

    elif aggregation == 'monthly':
        if aggregation_spec is None:
            aggregation_spec = range(1, 13)  # Default to all months (January to December)
        resampled = ds.sel(date=ds['date'].dt.month.isin(aggregation_spec)).resample(date='Y').mean()
        output_suffix = f"monthly_{'_'.join(map(str, aggregation_spec))}_{min_year}_{max_year}"
        process_and_save(resampled, input_nc_file, output_suffix, min_year, max_year)

    elif aggregation == 'seasonal':
        if aggregation_spec is None:
            aggregation_spec = ['DJF', 'MAM', 'JJA', 'SON']  # Default to all seasons
        for season in aggregation_spec:
            resampled = resample_seasonal(ds, season)
            output_suffix = f"seasonal_{season}_{min_year}_{max_year}"
            process_and_save(resampled, input_nc_file, output_suffix, min_year, max_year)

    elif aggregation == 'year_interval':
        if aggregation_spec is None:
            aggregation_spec = [1]  # Default to 1 year intervals (i.e., yearly)
        resampled = resample_by_year_interval(ds, aggregation_spec[0])
        output_suffix = f"year_interval_{aggregation_spec[0]}_{min_year}_{max_year}"
        process_and_save(resampled, input_nc_file, output_suffix, min_year, max_year)

    else:
        raise ValueError("Invalid aggregation. Choose 'yearly', 'monthly', 'seasonal', or 'year_interval'.")

def resample_seasonal(ds, season):
    # Define seasons
    seasons = {"DJF": [12, 1, 2], "MAM": [3, 4, 5], "JJA": [6, 7, 8], "SON": [9, 10, 11]}
    if season not in seasons:
        raise ValueError("Invalid season. Choose from 'DJF', 'MAM', 'JJA', 'SON'.")
    return ds.sel(date=ds['date'].dt.month.isin(seasons[season])).resample(date='Y').mean()

def resample_by_year_interval(ds, year_interval):
    # Resample by custom year intervals
    years = ds['date'].dt.year.values
    intervals = np.arange(years.min(), years.max() + 1, year_interval)
    return ds.groupby_bins('date', intervals).mean(dim='date')

def process_and_save(resampled, input_nc_file, output_suffix, min_year=None, max_year=None):
    # Convert temperature to Fahrenheit
    resampled['t2m'] = (resampled['t2m'] - 273.15) * 9 / 5 + 32  # Convert to Fahrenheit
    years = resampled['date'].dt.year.values
    start_year = min_year if min_year else years.min()
    end_year = max_year if max_year else years.max()

    # Calculate the total number of years in the range
    total_years = end_year - start_year

    year_mask = (years >= start_year) & (years <= end_year)
    t2m_data = resampled['t2m'].values[year_mask, :, :]

    lat, lon = resampled['latitude'].values, resampled['longitude'].values
    trend_array = np.zeros((len(lat), len(lon)), dtype=np.float32)

    for i in range(len(lat)):
        for j in range(len(lon)):
            pixel_time_series = t2m_data[:, i, j]
            if np.isnan(pixel_time_series).all():
                continue
            valid_idx = ~np.isnan(pixel_time_series)
            x = years[year_mask][valid_idx]
            y = pixel_time_series[valid_idx]
            slope = calculate_ols_trend_using_numpy(x, y)
            
            # Multiply slope by total years to calculate overall change over the period
            trend_array[i, j] = slope * total_years

    # Define the transform and metadata for the GeoTIFF
    transform = from_origin(np.min(lon), np.max(lat), np.abs(lon[1] - lon[0]), np.abs(lat[1] - lat[0]))

    meta = {
        'driver': 'GTiff',
        'height': trend_array.shape[0],
        'width': trend_array.shape[1],
        'count': 1,
        'dtype': 'float32',
        'crs': 'EPSG:4326',
        'transform': transform
    }

    # Define output filename and save GeoTIFF
    output_file_name = f"{Path(input_nc_file).stem}_ols_trend_{output_suffix}.tif"
    output_tif = Path(input_nc_file).with_name(output_file_name)

    with rasterio.open(output_tif, 'w', **meta) as dst:
        dst.write(trend_array, 1)

    print(f"GeoTIFF saved as {output_file_name}")

def calculate_ols_trend_using_numpy(x, y):
    # Calculate the slope using numpy's polyfit function
    slope, _ = np.polyfit(x, y, 1)
    return slope

# Example usage:
input_nc_file = r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\Monthly_single_l\data_0.nc"
process_ols_regression(input_nc_file, aggregation='monthly', aggregation_spec=[12], min_year=1950)


GeoTIFF saved as data_0_ols_trend_monthly_12_1950_None.tif


#### Process contour

In [ ]:
import arcpy
import pathlib

def process_conour(input_tif_file):
    raster_min = arcpy.Raster(input_tif_file).minimum
    raster_max = arcpy.Raster(input_tif_file).maximum
    ct_interval = ((raster_max-raster_min)/2)/10
    arcpy.ddd.Contour(input_tif_file, f"{pathlib.Path(input_tif_file).stem}_contour.shp", 
                      contour_interval=ct_interval, contour_type="CONTOUR_POLYGON")

input_tif_file = r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\Monthly_single_l\data_0_ols_trend_monthly_12_1940_2023.tif"
process_conour(input_tif_file)

#### Process cumulative Theil-Sen to make animation

In [ ]:
import xarray as xr
import numpy as np
from scipy.stats import theilslopes
import rasterio
from rasterio.transform import from_origin
from pathlib import Path

def process_cumulative_theil_sen_for_animation(input_nc_file, aggregation=5):
    """
    Calculate cumulative Theil-Sen slopes for temperature data (converted to Fahrenheit) over
    cumulative ranges starting from the baseline year, and save results as individual GeoTIFFs in a directory.

    Parameters:
    input_nc_file (str): Path to the NetCDF file with temperature data.
    aggregation (int): Number of years to step for cumulative ranges. Default is 5.

    Outputs:
    Individual GeoTIFF files with cumulative slopes saved in a directory.
    """
    ds = xr.open_dataset(input_nc_file, engine='netcdf4')
    ds['date'] = pd.to_datetime(ds['date'].values)
    yearly_data = ds.resample(date='Y').mean()
    t2m = (yearly_data['t2m'].values - 273.15) * 9 / 5 + 32
    lat, lon = yearly_data['latitude'].values, yearly_data['longitude'].values
    years = yearly_data['date.year'].values

    output_dir = Path(input_nc_file).parent / "cumulative_regres"
    output_dir.mkdir(exist_ok=True)

    for end_idx in range(aggregation, len(years) + 1, aggregation):
        time_range = f"{years[0]}-{years[end_idx - 1]}"
        t2m_subset = t2m[:end_idx]
        slope_array = np.full((len(lat), len(lon)), np.nan, dtype=np.float32)

        for i in range(len(lat)):
            for j in range(len(lon)):
                pixel_time_series = t2m_subset[:, i, j]
                if not np.any(np.isnan(pixel_time_series)):
                    slope_array[i, j] = theilslopes(pixel_time_series, np.arange(len(pixel_time_series)))[0]

        cumulative_raster = slope_array * end_idx
        transform = from_origin(np.min(lon), np.max(lat), np.abs(lon[1] - lon[0]), np.abs(lat[1] - lat[0]))
        meta = {
            'driver': 'GTiff',
            'height': cumulative_raster.shape[0],
            'width': cumulative_raster.shape[1],
            'count': 1,
            'dtype': 'float32',
            'crs': 'EPSG:4326',
            'transform': transform
        }

        output_file = output_dir / f"cumulative_{time_range}.tif"
        with rasterio.open(output_file, 'w', **meta) as dst:
            dst.write(cumulative_raster, 1)

        print(f"Saved GeoTIFF for {time_range} as {output_file}")

# Example usage:
input_nc_file = r"D:\UCalgary_Lectures\GEOG_683\Data_workspace\Monthly_single_l\data_0.nc"
process_cumulative_theil_sen_for_animation(input_nc_file, aggregation=5)
